In [1]:
import numpy as np
import pandas as pd

c:\Users\user\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\user\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
data_directory = 'data/'
file_names = [
    'Batch_corrected_Expression_Public_24Q4_subsetted.csv',
    'Damaging_Mutations_subsetted.csv', 
    'Harmonized_RPPA_CCLE_subsetted.csv',
    'Hotspot_Mutations_subsetted.csv', 
    'IC50_AUC_merged.csv', 
    'Metabolomics_subsetted.csv',
    'miRNA_Expression_subsetted.csv',
    'Omics_Absolute_CN_Gene_Public_24Q4_subsetted.csv'
    ]

In [3]:
IC50_AUC = pd.read_csv(f'{data_directory}{file_names[4]}')
IC50_AUC = IC50_AUC.rename(columns={"Unnamed: 0": "CellLineID"})

In [4]:
melted = IC50_AUC.melt(
    id_vars=["CellLineID"],  # 고정할 컬럼(세포주 ID)
    var_name="metric_drug",  # 가로였던 열 이름들이 녹아들어갈 컬럼
    value_name="value"       # 값
)
melted.head()

,CellLineID,metric_drug,value
0,ACH-000001,Drug sensitivity AUC (Sanger GDSC2) (+)-CAMPTO...,0.967187
1,ACH-000002,Drug sensitivity AUC (Sanger GDSC2) (+)-CAMPTO...,0.711866
2,ACH-000004,Drug sensitivity AUC (Sanger GDSC2) (+)-CAMPTO...,0.742350
3,ACH-000006,Drug sensitivity AUC (Sanger GDSC2) (+)-CAMPTO...,0.752250
4,ACH-000007,Drug sensitivity AUC (Sanger GDSC2) (+)-CAMPTO...,0.845466


In [5]:
import re

# 예: "Drug sensitivity AUC (Sanger GDSC2) CAMPTOTHECIN (GDSC2:1003)"
#     -> (AUC), (CAMPTOTHECIN), (1003)
pattern = r"(AUC|IC50).*?\((Sanger GDSC2)\)\s+(.*?)\s+\(GDSC2:(\d+)\)"

# capture group 설명
# (AUC|IC50) -> 1) AUC 혹은 IC50
# (.*?)      -> 3) 약물명 (최소 매칭)
# (\d+)      -> 4) GDSC2: 뒤에 오는 숫자


In [6]:
melted[["Metric", "_sanger", "DrugName", "DrugNumber"]] = melted["metric_drug"].str.extract(pattern)
#melted["DrugID"] = "GDSC2:" + melted["DrugNumber"]
new_melted = melted[['CellLineID', 'value', 'Metric','DrugNumber']]
new_melted.head()

,CellLineID,value,Metric,DrugNumber
0,ACH-000001,0.967187,AUC,1003
1,ACH-000002,0.711866,AUC,1003
2,ACH-000004,0.742350,AUC,1003
3,ACH-000006,0.752250,AUC,1003
4,ACH-000007,0.845466,AUC,1003


In [7]:
final_IC50_AUC = (
    new_melted
    .pivot(index=["CellLineID", "DrugNumber"], 
           columns="Metric", 
           values="value")
    #.reset_index()
)


In [8]:
final_IC50_AUC = final_IC50_AUC.dropna(subset=['AUC', 'IC50'], how='all')
final_IC50_AUC.columns.name = None
#final_IC50_AUC.rename(columns={'Metric': 'index'})
final_IC50_AUC

AUC     IC50
CellLineID DrugNumber                   
ACH-000001 1003        0.967187      NaN
           1004        0.666062      NaN
           1005        0.977907      NaN
           1007        0.922989      NaN
           1010        0.957586      NaN
...                         ...      ...
ACH-002317 2096        0.946713      NaN
           2107        0.926422      NaN
           2109        0.916274  3.25832
           2110        0.906540      NaN
           2170        0.996681      NaN

[98430 rows x 2 columns]

In [9]:
final_IC50_AUC.to_csv("temp.csv")

# Now, map DrugNumber and SMILES

In [10]:
IC50_AUC_final = pd.read_csv("temp.csv")

In [11]:
IC50_AUC_final.columns

Index(['CellLineID', 'DrugNumber', 'AUC', 'IC50'], dtype='object')

In [12]:
type(IC50_AUC_final["DrugNumber"][0])

numpy.int64

In [13]:
SMILES = pd.read_csv("./data/drugID_name_pubchem_smiles.csv")
type(SMILES["drugID"][0])

numpy.int64

In [14]:
df_merged = pd.merge(IC50_AUC_final, SMILES, left_on='DrugNumber', right_on='drugID', how='inner')


In [19]:
ic50_data = df_merged[['CellLineID', 'DrugNumber', 'smiles', 'IC50']].copy()
ic50_data['AUC'] = None  # IC50이 있는 행에서는 AUC를 NaN으로 설정
ic50_data = ic50_data.rename(columns={'IC50': 'IC50'})

auc_data = df_merged[['CellLineID', 'DrugNumber', 'smiles', 'AUC']].copy()
auc_data['IC50'] = None  # AUC가 있는 행에서는 IC50을 NaN으로 설정
auc_data = auc_data.rename(columns={'AUC': 'AUC'})

# 두 개의 데이터프레임을 합쳐서 한 행에 하나의 값만 있도록 변환
final_df = pd.concat([ic50_data, auc_data])

# 정렬 (순서 유지)
final_df = final_df.sort_values(by=['CellLineID', 'DrugNumber']).reset_index(drop=True)

final_df = final_df.dropna(subset=['AUC', 'IC50'], how='all').reset_index(drop=True)

# 결과 확인
final_df

C:\Users\user\AppData\Local\Temp\ipykernel_9844\4033451693.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([ic50_data, auc_data])


,CellLineID,DrugNumber,smiles,IC50,AUC
0,ACH-000001,1003,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...,NaN,0.967187
1,ACH-000001,1004,CCC1(CC2CC(C3=C(CCN(C2)C1)C4=CC=CC=C4N3)(C5=C(...,NaN,0.666062
2,ACH-000001,1005,N.N.[Cl-].[Cl-].[Pt+2],NaN,0.977907
3,ACH-000001,1007,CC1=C2C(C(=O)C3(C(CC4C(C3C(C(C2(C)C)(CC1OC(=O)...,NaN,0.922989
4,ACH-000001,1010,COC1=C(C=C2C(=C1)N=CN=C2NC3=CC(=C(C=C3)F)Cl)OC...,NaN,0.957586
...,...,...,...,...,...
117641,ACH-002317,2107,C1COCCN1C2=CC=C(C=C2)C3=C(C=NC=C3)C4=CC(=C(C(=...,NaN,0.926422
117642,ACH-002317,2109,CC1CN(CC(N1)C)C2=NC=C(C(=C2)C)C3=CC=C(C=C3)C4=...,3.25832,NaN
117643,ACH-002317,2109,CC1CN(CC(N1)C)C2=NC=C(C(=C2)C)C3=CC=C(C=C3)C4=...,NaN,0.916274
117644,ACH-002317,2110,C1CC(C1)NC2=NC=CC(=C2)C(=O)NCC(CN3CCC4=CC=CC=C...,NaN,0.906540


In [20]:
df_final = final_df[['CellLineID', 'DrugNumber', 'smiles', 'AUC', 'IC50']]
df_final.to_csv("./data/IC50_AUC_SMILES_final.csv", index=False)
df_final_train = final_df[['CellLineID', 'smiles', 'AUC', 'IC50']]
df_final_train.to_csv("./data/IC50_AUC_SMILES_final_train.csv", index=False)

In [21]:
df = pd.read_csv("./data/IC50_AUC_SMILES_final.csv")
df

,CellLineID,DrugNumber,smiles,AUC,IC50
0,ACH-000001,1003,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...,0.967187,NaN
1,ACH-000001,1004,CCC1(CC2CC(C3=C(CCN(C2)C1)C4=CC=CC=C4N3)(C5=C(...,0.666062,NaN
2,ACH-000001,1005,N.N.[Cl-].[Cl-].[Pt+2],0.977907,NaN
3,ACH-000001,1007,CC1=C2C(C(=O)C3(C(CC4C(C3C(C(C2(C)C)(CC1OC(=O)...,0.922989,NaN
4,ACH-000001,1010,COC1=C(C=C2C(=C1)N=CN=C2NC3=CC(=C(C=C3)F)Cl)OC...,0.957586,NaN
...,...,...,...,...,...
117641,ACH-002317,2107,C1COCCN1C2=CC=C(C=C2)C3=C(C=NC=C3)C4=CC(=C(C(=...,0.926422,NaN
117642,ACH-002317,2109,CC1CN(CC(N1)C)C2=NC=C(C(=C2)C)C3=CC=C(C=C3)C4=...,NaN,3.25832
117643,ACH-002317,2109,CC1CN(CC(N1)C)C2=NC=C(C(=C2)C)C3=CC=C(C=C3)C4=...,0.916274,NaN
117644,ACH-002317,2110,C1CC(C1)NC2=NC=CC(=C2)C(=O)NCC(CN3CCC4=CC=CC=C...,0.906540,NaN


In [22]:
df = pd.read_csv("./data/IC50_AUC_SMILES_final_train.csv")
df

,CellLineID,smiles,AUC,IC50
0,ACH-000001,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...,0.967187,NaN
1,ACH-000001,CCC1(CC2CC(C3=C(CCN(C2)C1)C4=CC=CC=C4N3)(C5=C(...,0.666062,NaN
2,ACH-000001,N.N.[Cl-].[Cl-].[Pt+2],0.977907,NaN
3,ACH-000001,CC1=C2C(C(=O)C3(C(CC4C(C3C(C(C2(C)C)(CC1OC(=O)...,0.922989,NaN
4,ACH-000001,COC1=C(C=C2C(=C1)N=CN=C2NC3=CC(=C(C=C3)F)Cl)OC...,0.957586,NaN
...,...,...,...,...
117641,ACH-002317,C1COCCN1C2=CC=C(C=C2)C3=C(C=NC=C3)C4=CC(=C(C(=...,0.926422,NaN
117642,ACH-002317,CC1CN(CC(N1)C)C2=NC=C(C(=C2)C)C3=CC=C(C=C3)C4=...,NaN,3.25832
117643,ACH-002317,CC1CN(CC(N1)C)C2=NC=C(C(=C2)C)C3=CC=C(C=C3)C4=...,0.916274,NaN
117644,ACH-002317,C1CC(C1)NC2=NC=CC(=C2)C(=O)NCC(CN3CCC4=CC=CC=C...,0.906540,NaN
